# Lab 4: Basic text preprocessing and topic modeling

The goal of this assignment is to get familiar with textual data analysis.


**Submission: submit via onq.** 


In [1]:
! pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 7.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=84594db314a43da5ad188811df52a6b6984467dda0fe1d9b77f3877b83375cbb
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [2]:
# Import needed libraries. E.g., pandas, sklearn, nltk, etc. 
import pandas as pd
import sklearn
import nltk
import numpy as np
import os
#import libraries
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as esw
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

import nltk
nltk.download('punkt')

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt

import string

# allow display of multiple outputs by running one code cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Get the current working directory
os.getcwd()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


'/content'

Task 1: The given dataset is a table containing questions about R on StackOverflow site. Your first task is to perform standard text prepossessing steps introduced in lectures for future tasks. You can perform analysis on title or body of the questions.

In [3]:
# TODO code for task 1
from google.colab import drive
drive.mount('/content/drive')
# os.chdir('/content/drive/MyDrive/Questions.csv')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/Questions.csv')
df.head()

,Id,OwnerUserId,CreationDate,Score,Title,Body
0,77434,14008.0,2008-09-16T21:40:29Z,171,How to access the last value in a vector?,<p>Suppose I have a vector that is nested in a...
1,79709,NaN,2008-09-17T03:39:16Z,3,Worse sin: side effects or passing massive obj...,<p>I have a function inside a loop inside a fu...
2,95007,15842.0,2008-09-18T17:59:19Z,56,Explain the quantile() function in R,<p>I've been mystified by the R quantile funct...
3,103312,NaN,2008-09-19T16:09:26Z,4,How to test for the EOF flag in R?,<p>How can I test for the <code>EOF</code> fla...
4,255697,1941213.0,2008-11-01T15:48:30Z,4,Is there an R package for learning a Dirichlet...,<p>I'm looking for a an <code>R</code> package...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189930 entries, 0 to 189929
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Id            189930 non-null  int64  
 1   OwnerUserId   188729 non-null  float64
 2   CreationDate  189930 non-null  object 
 3   Score         189930 non-null  int64  
 4   Title         189930 non-null  object 
 5   Body          189930 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 8.7+ MB


so there is no missing values

In [6]:
# Let's first check highly frequently appearing words on this corpus, excluding normal English stop-words 

def get_top_n_words(corpus, n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    
    get_top_n_words(["I love Python", "Python is a language programming", "Hello world", "I love the world"]) -> 
    [('python', 2),
     ('world', 2),
     ('love', 2),
     ('hello', 1),
     ('is', 1),
     ('programming', 1),
     ('the', 1),
     ('language', 1)]
    """
    vec = CountVectorizer(stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)      
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]     
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)    
    return words_freq[:n]

common_words = get_top_n_words(df['Title'], 50)
for word, freq in common_words:
    print(word, freq)

data 29767
using 18842
function 13756
column 11865
frame 10747
error 10601
values 10520
plot 9944
multiple 8275
package 7964
columns 7907
table 7803
list 7556
matrix 6755
ggplot2 6514
value 6411
shiny 6369
dataframe 6031
rows 5987
file 5975
variable 5918
loop 5763
vector 5555
based 5187
time 5109
row 5002
create 4732
different 4601
use 4514
ggplot 4448
variables 4105
string 3911
number 3786
add 3443
axis 3292
object 3261
output 3138
text 3048
names 3034
date 3034
new 2976
dplyr 2857
change 2828
convert 2791
model 2697
code 2648
line 2645
set 2580
specific 2579
files 2577


In [7]:
# let's use standard english stop-words first, later we would see if we need to filter specific stopwords (high-frequent words with no discriminative power)
cachedStopWords = ["using", 'use', 'change', 'convert'] + list(esw)      #esw: English Stop Words
lemmatizer=WordNetLemmatizer()
#ps = PorterStemmer()

def lemmatize_article(sentence):
    sentence = word_tokenize(sentence)
    res = ''
    for word, tag in pos_tag(sentence):
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        word = lemmatizer.lemmatize(word, wntag) if wntag else word
        res += word + ' '
    return res
    
def remove_stop_words(sentence):
    return ' '.join([word for word in sentence.split() if word not in cachedStopWords])
    
def remove_short(sentence):
    return ' '.join([word for word in sentence.split() if len(word) >= 3])
    
def remove_digits(sentence):
    return ' '.join([i for i in sentence.split() if not i.isdigit()])
    
def preprocess(all_texts):
    all_texts = list(map(lambda x: x.lower(), all_texts))
    all_texts = list(map(lambda x: x.translate(str.maketrans('', '', string.punctuation)), all_texts))
    all_texts = list(map(lambda x: lemmatize_article(x), all_texts))
    all_texts = list(map(lambda x: x.strip(), all_texts))
    all_texts = list(map(lambda x: remove_stop_words(x), all_texts))
    all_texts = list(map(lambda x: remove_short(x), all_texts))
    all_texts = list(map(lambda x: remove_digits(x), all_texts))
    return all_texts
    

In [8]:
# preprocess title
all_titles_processed = preprocess(df['Title'])
all_titles_processed


['access value vector',
 'worse sin effect pass massive object',
 'explain quantile function',
 'test eof flag',
 'package learn dirichlet prior count data',
 'optimization package',
 'think vector',
 'vectorize thinking vector operation',
 'compiled language',
 'filter data',
 'operate time interval like',
 'whats easiest way install 100s file visual studio setup project',
 'csv file multiple timeseries',
 'quotelike operator like perls',
 'write gui brcapro cancer gene risk calculation engine',
 'sort axis barplot',
 'best practice handle time',
 'best way avoid pass data frame',
 'function',
 'remove element list',
 'cumulative plot xaxis',
 'column store compare column base database',
 'expensive compute eigenvalue matrix',
 'color edge draw rects correctly dendrogram',
 'random forest variable importance',
 'plot titleslabels',
 'row index contain maximum value',
 'calculate average',
 'suppress null device output batch mode',
 'whats best way script command line terminal',
 'func

Task 2: Using existing libraries, such as gensim https://radimrehurek.com/gensim/ to learn word embeddings from the prepossessed text from previous step. At the end of this step, you should save the learned word embeddings in a file.

In [9]:
# TODO code for task 2
# let's just try with 10 categories
tf_vectorizer = CountVectorizer(max_df = 0.5,           #terms occur in more than 50% of documents, terms occur less than 10 times --> ignore
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(all_titles_processed)


In [10]:
df_tf = pd.DataFrame(dtm_tf.toarray(), columns = tf_vectorizer.get_feature_names())
df_tf.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,1st,2nd,2way,2x2,32bit,3dimensional,3rd,5th,64bit,abbreviate,...,zerolength,zip,zipcode,zipcodes,zipped,zone,zoo,zoom,zscore,zscores
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df_tf.to_csv('wordembeddings_title.csv', index=False)

Task 3: Perform topic analysis on the prepossessed textual data. Briefly specify how you pick the number of topics. 

Present your findings (the final set of topics you extracted, the popularity of each topic). 

In [12]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tf.fit(dtm_tf)

pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

LatentDirichletAllocation(random_state=0)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.256178  0.062181       1        1  15.818634
1      0.208704  0.077797       2        1  10.202450
5     -0.018782 -0.327675       3        1   9.963376
4      0.154357  0.163635       4        1   9.778644
7     -0.281294  0.090722       5        1   9.665288
6     -0.250719  0.154856       6        1   9.654706
8     -0.022560 -0.271183       7        1   9.283125
2     -0.139270  0.160607       8        1   9.216876
0      0.153666  0.048277       9        1   8.752369
3     -0.060281 -0.159218      10        1   7.664533, topic_info=          Term          Freq         Total Category  logprob  loglift
572     column  18033.000000  18033.000000  Default  30.0000  30.0000
825       data  22374.000000  22374.000000  Default  29.0000  29.0000
1432  function  15397.000000  15397.000000  Default  28.0000  28.0000
2794      plot  14169.000000  14169.000000  Default  27.0000  27.0000
2667   package   8583.000000   8583.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
3458      save    505.885298   1548.360994  Topic10  -4.8716   1.4499
4287     value    803.375234  15754.395974  Topic10  -4.4091  -0.4075
767     create    573.014044   7453.233973  Topic10  -4.7470   0.0031
4291  variable    543.591317   9724.302749  Topic10  -4.7997  -0.3156
48         add    470.047028   5291.805472  Topic10  -4.9451   0.1475

[619 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
26        1  0.768305  accord
26        2  0.015298  accord
26        3  0.061192  accord
26        4  0.003400  accord
26        7  0.149581  accord
...     ...       ...     ...
4472      3  0.987518   yaxis
4472      7  0.011299   yaxis
4473      2  0.998232    year
4482     10  0.988906    ylim
4498      2  0.997382     zoo

[1179 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 2, 6, 5, 8, 7, 9, 3, 1, 4])

In [13]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=7, random_state=0)
lda_tf.fit(dtm_tf)

pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

LatentDirichletAllocation(n_components=7, random_state=0)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.202030 -0.136147       1        1  16.987481
4     -0.179693 -0.214480       2        1  16.208229
6      0.284813 -0.064491       3        1  15.204977
5     -0.046979  0.344631       4        1  15.038968
2      0.218641 -0.045828       5        1  13.741173
1     -0.168228  0.123579       6        1  11.413375
0      0.093476 -0.007263       7        1  11.405796, topic_info=          Term          Freq         Total Category  logprob  loglift
825       data  22341.000000  22341.000000  Default  30.0000  30.0000
4287     value  16068.000000  16068.000000  Default  29.0000  29.0000
2794      plot  13562.000000  13562.000000  Default  28.0000  28.0000
572     column  18648.000000  18648.000000  Default  27.0000  27.0000
1432  function  15241.000000  15241.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
3563  sequence    518.539174   1010.578800   Topic7  -5.2444   1.5038
3232   replace    606.865611   2258.924240   Topic7  -5.0871   0.8567
685    contain    490.283124   1155.776444   Topic7  -5.3004   1.3135
1294      fail    466.101255    882.505364   Topic7  -5.3510   1.5327
3744  specific    464.775283   2503.663603   Topic7  -5.3538   0.4871

[428 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
48        1  0.346741    add
48        2  0.095334    add
48        4  0.548942    add
48        6  0.003268    add
48        7  0.005382    add
...     ...       ...    ...
4463      6  0.998199    xts
4472      4  0.995701  yaxis
4472      6  0.002359  yaxis
4473      6  0.998330   year
4498      6  0.998106    zoo

[757 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 7, 6, 3, 2, 1])

In [14]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=5, random_state=0)
lda_tf.fit(dtm_tf)

pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

LatentDirichletAllocation(n_components=5, random_state=0)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.223406  0.139159       1        1  21.825114
1      0.013741 -0.360923       2        1  20.927575
3     -0.249636  0.030892       3        1  20.800344
4     -0.200388  0.142887       4        1  19.667601
0      0.212876  0.047986       5        1  16.779366, topic_info=         Term          Freq         Total Category  logprob  loglift
825      data  22629.000000  22629.000000  Default  30.0000  30.0000
4287    value  16403.000000  16403.000000  Default  29.0000  29.0000
2794     plot  13608.000000  13608.000000  Default  28.0000  28.0000
572    column  19073.000000  19073.000000  Default  27.0000  27.0000
2667  package   8949.000000   8949.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
4306   vector   2375.452076   6984.152834   Topic5  -4.1085   0.7066
1194    error   2404.639901  10938.263825   Topic5  -4.0963   0.2702
2123     load    872.368131   1461.414763   Topic5  -5.1102   1.2691
2237    match    969.141620   2546.574358   Topic5  -5.0050   0.8189
3209   remove    993.005177   3142.321550   Topic5  -4.9807   0.6330

[317 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
48        1  0.000379     add
48        2  0.460813     add
48        3  0.523720     add
48        4  0.001326     add
48        5  0.013832     add
...     ...       ...     ...
4434      4  0.001206   write
4434      5  0.687782   write
4444      2  0.999250   xaxis
4461      3  0.997041  xtable
4472      2  0.997487   yaxis

[556 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 4, 5, 1])